# Twitter Data Exploration & Analysis

In [1]:
import os
import json
import pandas
import random
import bisect # Bin search

# Data stores
from pymongo import MongoClient
import psycopg # Postgres
import redis

In [2]:
# Utility function for getting data
def getDataDir():
    homeDir=os.getenv("HOME")
    return os.path.join(homeDir, "data")

getDataDir()

'/home/jovyan/data'

In [3]:
# Find "leftmost" value exactly equal to X. If not found
# element doesn't exist
def fnBinSearch(lsSorted, val):
    index = bisect.bisect_left(lsSorted, val)
    if index != len(lsSorted) and lsSorted[index] == val:
        return index
    return None

## Part 1 - Parsing & Exploring the Data

First step is to read the data from file, one record at a time and then
gather statistics and information about the fields. We do this 1 record
at a time to avoid loading everything into memory.

In [4]:
def fnGetDataFromJSON(sLine):
    data = None
    try:
        data = json.loads(sLine)
    except Exception as e:
        print("Parsing error:", e)
    
    return data

In [5]:
# Sample data should be in this directory
filepath = os.path.join(getDataDir(), "corona-out-2")

In [6]:
def fnInitFieldDic(name):
    return {"name": name, "count": 1, "nonNullCount": 0, "conflictTypesCount": 0, 
            "type": None, "subfields": {}} 

fnInitFieldDic("fName")

{'name': 'fName',
 'count': 1,
 'nonNullCount': 0,
 'conflictTypesCount': 0,
 'type': None,
 'subfields': {}}

In [7]:
# Global Fields with complex(dictionary) data should be parsed
COMPLEX_FIELD_LIST = ["user", "place", "retweeted_status", "quoted_status", 
                    "entities", "extended_tweet", "extented_entities" ]

In [8]:
# Simple function that prints out a JSON in a slightly more readable format with indents
def fnPreview(currentRecord, processArgs):
    print(json.dumps(currentRecord, indent=4))
    return None

In [9]:
# Return the JSON objects as a list
def fnGetList(currentRecord, processArgs):
    lsRecords = processArgs["recordList"]
    lsRecords.append(currentRecord)
    return None

In [10]:
def fnProcessObj(data, fieldDic):
    if data != None:
        # Get the fields from the json.
        for field in data:
            # Create a record of this field
            if not field in fieldDic.keys():
                fieldDic[field] = fnInitFieldDic(field)
            else:
                fieldRec = fieldDic[field]
                fieldRec["count"] = fieldRec["count"] + 1

            # Differentiate NULL entries and extract type if not known
            value = data[field]
            if value is not None:
                fieldType = type(value)
                fieldRec = fieldDic[field]
                # Count the conflicts if type is not consistent. Data consistency. 
                if fieldType is not None and fieldRec["type"] is not None:
                    if fieldType != fieldRec["type"]:
                        fieldRec["conflictTypesCount"] = fieldRec["conflictTypesCount"] + 1

                # Save last type encountered
                if fieldType is not None:
                    fieldRec["nonNullCount"] = fieldRec["nonNullCount"] + 1
                    fieldRec["type"] = fieldType

In [11]:
# Collect basic stats
def fnGetStats(currentRecord, processArgs):
    fieldDic = processArgs["fieldDic"]
    
    # Calculate field and stats
    fnProcessObj(currentRecord, fieldDic)
    processArgs["numProcessed"] = processArgs["numProcessed"] + 1

    # For certain complex fields get the subfields (see list above)
    for complexField in COMPLEX_FIELD_LIST:
        if complexField in currentRecord:
            # Get the value
            complexFieldValue = currentRecord[complexField]
            # Get the record for that field
            fieldRec = fieldDic[complexField]
            if complexFieldValue != None:
                fnProcessObj(complexFieldValue, fieldRec["subfields"])
    return None

In [12]:
# Print stats
def fnPrintStats(parent, fields):
    print("")
    print("Field States: ", parent)
    print("")

    # Print out header for fields/objects in tweets and count. Fields 1:1 (required) or sparse
    # Kind of like a data frame ...
    print("{name:32s} {count:<6s} {nonNullCount:<9s} {conflictTypesCount:<12s} {fType:>16s}".format(
        name="Name", count="Count", nonNullCount="Not Null", conflictTypesCount="Type Conflict", 
        fType="Field Type"))
    
    # For each field 
    for field in fields.keys():
        fieldRec = fields[field]
        fieldName = fieldRec["name"]
        fieldCount = fieldRec["count"]
        fieldNonNullCount = fieldRec["nonNullCount"]
        fieldConflictTypesCount = fieldRec["conflictTypesCount"] 
        fieldType = fieldRec["type"]

        if fieldType is not None:
            fieldType = fieldType.__name__
        else:
            fieldType = "Undefined"

        print("{name:32s} {count:<6d} {nonNullCount:<9d} {conflictTypesCount:<12d} {fType:>16s}".format(
            name=fieldName, count=fieldCount, nonNullCount=fieldNonNullCount, 
            conflictTypesCount=fieldConflictTypesCount, fType=fieldType))

In [13]:
# Somewhat generalized reader function that iterates over all the records, or a subset, sample
# and then calls the process function to do something useful. 
def fnGetRecords(filepath, fnProcessRecord, iFrom=None, iTo=None, lsSample=None, bVerbose=False, processArgs=None):
    
    iRecord = 0

    # Read JSON sample data with tweats
    try:
        print("Reading from ", filepath)

        with open(filepath, "r") as sampleFile:
            # Lets get it one line at a time to avoid loading everything into memory
            for sLine in sampleFile:
                # Ignore whitespaces
                if not sLine.isspace():
                    # Limit to a range
                    if not iFrom is None and iRecord < iFrom:
                        iRecord = iRecord + 1
                        continue
                    if not iTo is None and iRecord >= iTo:
                        break
                    # Or fetch specific records (based on a random sample)
                    if not lsSample is None and fnBinSearch(lsSample,iRecord) is None:
                        iRecord = iRecord + 1
                        continue
                    
                    if bVerbose:
                        print("Record", iRecord, ":")
                    
                    data = fnGetDataFromJSON(sLine)
                    # Got a data object
                    if not data is None:
                        fnProcessRecord(data, processArgs)
                                    
                    elif bVerbose:
                        print("Record is undefined or not parsed")
                    
                    # Keep track of all non empty lines being processed. Assume correspond to JSON
                    # top-level object
                    iRecord = iRecord + 1   
    except Exception as e:
        print("Error while reading JSON records from memory", e)
        
    return iRecord 

### Preview

First, let's get a snapshot of the JSON data. Defined a function that iterates over the dataset one
record at a time to simulate the "streaming" process. For the 1st pass, let's get the number of records
and

We use the prettify function in the <b>json</b> package which indents fields and exposes some of the hierarchy. 
The advantage of this approach over simply loading into a dataframe is it's more memory efficient and scalable 
with large datasets.

In [14]:
# Let's sneak a peek at the first record and get the number of records
numScanned = fnGetRecords(filepath, fnPreview, 0, 1, None, True)

Reading from  /home/jovyan/data/corona-out-2
Record 0 :
{
    "created_at": "Sun Apr 12 18:27:25 +0000 2020",
    "id": 1249403767180668930,
    "id_str": "1249403767180668930",
    "text": "RT @nuffsaidny: wishing death on people is weirdo behavior.",
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>",
    "truncated": false,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "in_reply_to_screen_name": null,
    "user": {
        "id": 1242817830946508801,
        "id_str": "1242817830946508801",
        "name": "juwelz v",
        "screen_name": "juwelz_v",
        "location": "Lower East Side, Manhattan",
        "url": "https://linktr.ee/juwelzv",
        "description": "Event Lyfe LLC .. Brand Ambassador: #visionarysociety Music Manager: @ssizzzlee & @tmoneybeatsbang Production\ud83c\udfa5\ud83d\udcf8 #BringNyCultureBack \ud83c\udf0a\

In [15]:
print("Scanned ", numScanned, " tweet(s)")

Scanned  1  tweet(s)


### Random Sample

This snippet samples the file and gets a random subset of data records. Again the idea is that
we're processing a potentially infinite (or very large) stream of records and these methods avoid
loading all the data in memory.

The conversion to Panda frames allows for easier viewing and querying of the data.

In [16]:
# Get a sample, 30 random records, from the 1st 1000 tweets to test the logic.
lsSample = random.sample(range(0, 1000), 30)
lsSample.sort()

# Arguments for the processing function
processArgs = {"recordList": []}
numScanned = fnGetRecords(filepath, fnGetList, 0, 1000, lsSample, False, processArgs)
print("Scanned ", numScanned, " records") 

Reading from  /home/jovyan/data/corona-out-2
Scanned  1000  records


In [17]:
# Convert list to data frame
df = pandas.DataFrame.from_records(processArgs["recordList"])

# Let's see the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   created_at                 30 non-null     object 
 1   id                         30 non-null     int64  
 2   id_str                     30 non-null     object 
 3   text                       30 non-null     object 
 4   source                     30 non-null     object 
 5   truncated                  30 non-null     bool   
 6   in_reply_to_status_id      7 non-null      float64
 7   in_reply_to_status_id_str  7 non-null      object 
 8   in_reply_to_user_id        7 non-null      float64
 9   in_reply_to_user_id_str    7 non-null      object 
 10  in_reply_to_screen_name    7 non-null      object 
 11  user                       30 non-null     object 
 12  geo                        0 non-null      object 
 13  coordinates                0 non-null      object 
 

In [18]:
df.head(5)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,possibly_sensitive,display_text_range,extended_tweet,extended_entities
0,Sun Apr 12 18:27:28 +0000 2020,1249403777804840962,1249403777804840962,RT @MADANI_SMS: R A M A D A N 🌙\nI S\n...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1586716048085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sun Apr 12 18:27:30 +0000 2020,1249403785849561091,1249403785849561091,RT @Andersblues: Bengan röstade väl Sd så ... ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,sv,1586716050003,1.249275e+18,1249274518125907968,{'created_at': 'Sun Apr 12 09:53:50 +0000 2020...,"{'url': 'https://t.co/46EFb6Dapq', 'expanded':...",False,NaN,NaN,NaN
2,Sun Apr 12 18:27:33 +0000 2020,1249403798612672513,1249403798612672513,@Uroosakhatti @alirazavohra Corona is Temporar...,"<a href=""http://twitter.com/download/android"" ...",False,1.249014e+18,1249013634380021762,1.085529e+18,1085529191041372160,...,in,1586716053046,NaN,NaN,NaN,NaN,NaN,"[28, 85]",NaN,NaN
3,Sun Apr 12 18:27:33 +0000 2020,1249403800617725957,1249403800617725957,Spahns Schuld keineswegs spanklein\nEtwas Kuns...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,de,1586716053524,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN
4,Sun Apr 12 18:27:34 +0000 2020,1249403804144918533,1249403804144918533,"RT @Karabo_Mokgoko: Yoh, Corona you won. Let u...","<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,en,1586716054365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Part 2 - Statistics 

Now let's collect some statistics. Once again we can get similar output by just using the
Panda dataframe but that would require loading the whole data set in memory. Instead reuse
the reader to collect stats one record at a time focusing on those fields which might have
interesting information to parse.

Iterate over all the records in the file and not just the ones which were samples from part 1 
using the function defined above. Selection of tweet fields which are themselves complex objects
are also parsed.

The key questions we need to answer are as follows:

* Which fields from the data streams do we keep and why?
* Do we store the fields as separate entities/tables?
* Do we store in the relational database (Postgres) or NoSQL (Mongo)? If so why?
* What needs to be cached (in Redis)?

In [19]:
processArgs = {"numProcessed": 0, "fieldDic": {}}
totalScanned = fnGetRecords(filepath, fnGetStats, None, None, None, False, processArgs)
print("Total Scanned ", totalScanned, " tweets")
print("Processed ", processArgs["numProcessed"], " tweets")      

Reading from  /home/jovyan/data/corona-out-2
Total Scanned  18518  tweets
Processed  18518  tweets


### Tweets

Starting with the top level object, tweets, stats show there are some fields which are clearly 1:1, 
i.e. every tweet has a non NULL record for the following.

* created_at: when the tweet has been created
* id/id_str: numerical and string representation of the ID. 
* text: the actual tweet itself. Potentially truncated to 140 characters
* user: who originated the tweet
* stats: quote/reply/retweet/favorite/entities/

In [20]:
tweetFields = processArgs["fieldDic"]
fnPrintStats("tweets", tweetFields)


Field States:  tweets

Name                             Count  Not Null  Type Conflict       Field Type
created_at                       18518  18518     0                         str
id                               18518  18518     0                         int
id_str                           18518  18518     0                         str
text                             18518  18518     0                         str
source                           18518  18518     0                         str
truncated                        18518  18518     0                        bool
in_reply_to_status_id            18518  2528      0                         int
in_reply_to_status_id_str        18518  2528      0                         str
in_reply_to_user_id              18518  2734      0                         int
in_reply_to_user_id_str          18518  2734      0                         str
in_reply_to_screen_name          18518  2734      0                         str
user           

### Complex Fields in Tweets

There are a number of fields in the tweet that are complex objects. The first one from the list
above is the users (creator). 

In [21]:
fnPrintStats("user", tweetFields["user"]["subfields"])


Field States:  user

Name                             Count  Not Null  Type Conflict       Field Type
id                               18518  18518     0                         int
id_str                           18518  18518     0                         str
name                             18518  18518     0                         str
screen_name                      18518  18518     0                         str
location                         18518  11456     0                         str
url                              18518  3974      0                         str
description                      18518  14608     0                         str
translator_type                  18518  18518     0                         str
protected                        18518  18518     0                        bool
verified                         18518  18518     0                        bool
followers_count                  18518  18518     0                         int
friends_count    

In [22]:
fnPrintStats("place", tweetFields["place"]["subfields"])


Field States:  place

Name                             Count  Not Null  Type Conflict       Field Type
id                               273    273       0                         str
url                              273    273       0                         str
place_type                       273    273       0                         str
name                             273    273       0                         str
full_name                        273    273       0                         str
country_code                     273    273       0                         str
country                          273    273       0                         str
bounding_box                     273    273       0                        dict
attributes                       273    273       0                        dict


In [23]:
fnPrintStats("retweeted_status", tweetFields["retweeted_status"]["subfields"])


Field States:  retweeted_status

Name                             Count  Not Null  Type Conflict       Field Type
created_at                       11159  11159     0                         str
id                               11159  11159     0                         int
id_str                           11159  11159     0                         str
text                             11159  11159     0                         str
source                           11159  11159     0                         str
truncated                        11159  11159     0                        bool
in_reply_to_status_id            11159  687       0                         int
in_reply_to_status_id_str        11159  687       0                         str
in_reply_to_user_id              11159  741       0                         int
in_reply_to_user_id_str          11159  741       0                         str
in_reply_to_screen_name          11159  741       0                         str
user 

In [24]:
fnPrintStats("quoted_status", tweetFields["quoted_status"]["subfields"])


Field States:  quoted_status

Name                             Count  Not Null  Type Conflict       Field Type
created_at                       2994   2994      0                         str
id                               2994   2994      0                         int
id_str                           2994   2994      0                         str
text                             2994   2994      0                         str
source                           2994   2994      0                         str
truncated                        2994   2994      0                        bool
in_reply_to_status_id            2994   191       0                         int
in_reply_to_status_id_str        2994   191       0                         str
in_reply_to_user_id              2994   205       0                         int
in_reply_to_user_id_str          2994   205       0                         str
in_reply_to_screen_name          2994   205       0                         str
user    

In [25]:
fnPrintStats("entities", tweetFields["entities"]["subfields"])


Field States:  entities

Name                             Count  Not Null  Type Conflict       Field Type
hashtags                         18518  18518     0                        list
urls                             18518  18518     0                        list
user_mentions                    18518  18518     0                        list
symbols                          18518  18518     0                        list
media                            1084   1084      0                        list


In [26]:
fnPrintStats("extended_tweet", tweetFields["extended_tweet"]["subfields"])


Field States:  extended_tweet

Name                             Count  Not Null  Type Conflict       Field Type
full_text                        2912   2912      0                         str
display_text_range               2912   2912      0                        list
entities                         2912   2912      0                        dict
extended_entities                474    474       0                        dict


In [27]:
fnPrintStats("extended_entities", tweetFields["extended_entities"]["subfields"])


Field States:  extended_entities

Name                             Count  Not Null  Type Conflict       Field Type


### Data Breakdown by Field

As shown below, we can differentiate between fields that are 1:1, and of simpler types with more
complex data types that may have many optional fields. In addition to stats, we should identify
which data entities are relatively static and structured, such as <b>user</b> and <b>location</b> vs
the <b>retweets</b> which may contain changing attribution.

## Part 3 - Tweet Fields

There are a number of interesting tools for visualizing Pandas including QGrid and Panda GUI. However,
neither of these add ons seem to be compatible with the latest version of Jupyter and/or Jupyter Hub.
Let's take a low tech approach which once again works with a larger sample of the data so we can explore
specific facets and odd attribution.

### Larger Sample

Given the size of the dataset and the counts above, will use a sample of 2000 records picked at random
from the entire dataset. The following snippets show samples of the records: tweets, users, places, retweets
quotes and entities.

In [28]:
lsLargerSample = random.sample(range(0, totalScanned), 2000)
lsLargerSample.sort()

# Arguments for the processing function
processArgs = {"recordList": []}
fnGetRecords(filepath, fnGetList, 0, totalScanned, lsLargerSample, False, processArgs)

Reading from  /home/jovyan/data/corona-out-2


18518

In [29]:
# Convert list to data frame
dfTweets = pandas.DataFrame.from_records(processArgs["recordList"])

len(dfTweets)

2000

### Common Tweet Fields

What are common and useful tweet fields? There are some obvious ones that should be kept including creation timestamp, ID, in reply to and in reply to user. 

In [30]:
dfTweets.head(5)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,retweeted_status,display_text_range,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities,possibly_sensitive,extended_tweet,withheld_in_countries
0,Sun Apr 12 18:27:26 +0000 2020,1249403771261722624,1249403771261722624,"#Bo der Osterhase hat nicht Corona, er ist gek...","<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sun Apr 12 18:27:27 +0000 2020,1249403775632076801,1249403775632076801,RT @detikcom: Seorang pria tiba-tiba tergeleta...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,{'created_at': 'Sun Apr 12 16:14:57 +0000 2020...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sun Apr 12 18:27:27 +0000 2020,1249403775812612096,1249403775812612096,Eu q lute https://t.co/lboqsO4MWp,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,NaN,"[0, 9]",1.249353e+18,1249352887840505867,{'created_at': 'Sun Apr 12 15:05:14 +0000 2020...,"{'url': 'https://t.co/0ogc9Xj7Jp', 'expanded':...","{'media': [{'id': 1249403764035006465, 'id_str...",False,NaN,NaN
3,Sun Apr 12 18:27:28 +0000 2020,1249403778136256513,1249403778136256513,"In Turkey, there are 300 thousand prisoners an...","<a href=""http://twitter.com/download/android"" ...",True,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'full_text': 'In Turkey, there are 300 thousa...",NaN
4,Sun Apr 12 18:27:28 +0000 2020,1249403779906035718,1249403779906035718,RT @ChristWamea: Pemimpin yg melayani rakyatny...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,{'created_at': 'Sun Apr 12 11:09:21 +0000 2020...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Grab the other complex records (objects)
dfUser = pandas.DataFrame.from_records(dfTweets["user"])
dfUser.head(5)

,id,id_str,name,screen_name,location,url,description,translator_type,protected,verified,...,profile_text_color,profile_use_background_image,profile_image_url,profile_image_url_https,default_profile,default_profile_image,following,follow_request_sent,notifications,profile_banner_url
0,1171484224244744198,1171484224244744198,Elfpunkt,meElfpunkt,None,None,Mal 1 mal 0 mal Tilt,none,False,False,...,333333,True,http://abs.twimg.com/sticky/default_profile_im...,https://abs.twimg.com/sticky/default_profile_i...,True,False,None,None,None,NaN
1,1013800044695179265,1013800044695179265,DavidGultomteknisilistrik,ParmonanganRaja,"DKI Jakarta, Indonesia",None,hidup penuh syukur dan rahmat 😇😇😇😇😇😇😇,none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1218719493...,https://pbs.twimg.com/profile_images/121871949...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/10138000...
2,1138436390897803265,1138436390897803265,fairy Tae ʚïɞ,umiiau,None,None,@BTS_twt 💜,none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1249203363...,https://pbs.twimg.com/profile_images/124920336...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/11384363...
3,755762321440514049,755762321440514049,CHN,ChnBba,Australia,None,None,none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1214687604...,https://pbs.twimg.com/profile_images/121468760...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/75576232...
4,1207221409144922112,1207221409144922112,sedang tidak baik baik saja,godelgila99,None,None,*waiting for your attention me*,none,False,False,...,333333,True,http://pbs.twimg.com/profile_images/1236661606...,https://pbs.twimg.com/profile_images/123666160...,True,False,None,None,None,https://pbs.twimg.com/profile_banners/12072214...


In [52]:
dfPlace = pandas.json_normalize(dfTweets["place"])
dfPlace = dfPlace[dfPlace["id"].notna()]
dfPlace.head(5)

,id,url,place_type,name,full_name,country_code,country,bounding_box.type,bounding_box.coordinates
114,317fcc4b21a604d5,https://api.twitter.com/1.1/geo/id/317fcc4b21a...,city,New Delhi,"New Delhi, India",IN,India,Polygon,"[[[76.84252, 28.397657], [76.84252, 28.879322]..."
196,00cc0d5640394308,https://api.twitter.com/1.1/geo/id/00cc0d56403...,admin,Punjab,"Punjab, Pakistan",PK,Pakistan,Polygon,"[[[69.328873, 27.708226], [69.328873, 34.01998..."
248,b7d3c12268abd20e,https://api.twitter.com/1.1/geo/id/b7d3c12268a...,city,Düsseldorf,"Düsseldorf, Deutschland",DE,Deutschland,Polygon,"[[[6.688728, 51.124357], [6.688728, 51.352537]..."
302,4ec01c9dbc693497,https://api.twitter.com/1.1/geo/id/4ec01c9dbc6...,admin,Florida,"Florida, USA",US,United States,Polygon,"[[[-87.634643, 24.396308], [-87.634643, 31.001..."
305,0006d55513b0c159,https://api.twitter.com/1.1/geo/id/0006d55513b...,city,Fairburn,"Fairburn, GA",US,United States,Polygon,"[[[-84.626791, 33.509481], [-84.626791, 33.592..."


In [56]:
dfRetweetStatus = pandas.json_normalize(dfTweets["retweeted_status"])
dfRetweetStatus = dfRetweetStatus[dfRetweetStatus["id"].notna()]
dfRetweetStatus.head(5)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,quoted_status.place.id,quoted_status.place.url,quoted_status.place.place_type,quoted_status.place.name,quoted_status.place.full_name,quoted_status.place.country_code,quoted_status.place.country,quoted_status.place.bounding_box.type,quoted_status.place.bounding_box.coordinates,withheld_in_countries
1,Sun Apr 12 16:14:57 +0000 2020,1.249370e+18,1249370429237661696,Seorang pria tiba-tiba tergeletak di trotoar M...,"<a href=""https://www.echobox.com"" rel=""nofollo...",True,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sun Apr 12 11:09:21 +0000 2020,1.249294e+18,1249293523490205696,Pemimpin yg melayani rakyatnya dengan hati pas...,"<a href=""http://twitter.com/download/android"" ...",True,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Sun Apr 12 02:30:02 +0000 2020,1.249163e+18,1249162832589008896,Vocês tem noção que a ITÁLIA tá pedindo prós I...,"<a href=""http://twitter.com/download/android"" ...",True,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Sun Apr 12 14:23:05 +0000 2020,1.249342e+18,1249342279845679104,Kerala had arnd 300 people who participated in...,"<a href=""http://twitter.com/download/android"" ...",True,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,Wed Apr 08 03:55:20 +0000 2020,1.247735e+18,1247734748534865920,"Yuk Sahabat, kita terapkan protokol keluar rum...","<a href=""http://twitter.com/download/android"" ...",True,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
dfQuotedStatus = pandas.json_normalize(dfTweets["quoted_status"])
dfQuotedStatus = dfQuotedStatus[dfQuotedStatus["id"].notna()]
dfQuotedStatus.head(5)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,quoted_status_id_str,place.id,place.url,place.place_type,place.name,place.full_name,place.country_code,place.country,place.bounding_box.type,place.bounding_box.coordinates
2,Sun Apr 12 15:05:14 +0000 2020,1.249353e+18,1249352887840505867,BigHit chefia pelo amor de Deus a gente ta em ...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Sun Apr 12 15:20:43 +0000 2020,1.249357e+18,1249356782088192001,بین الاقوامی برادری، اقوام متحدہ کے سیکرٹری جن...,"<a href=""http://twitter.com/download/android"" ...",True,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,Sun Apr 12 18:12:06 +0000 2020,1.249400e+18,1249399912023773185,"VIDEO : Special Report | कोरोनाचं संकट गंभीर, ...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Sun Apr 12 04:58:49 +0000 2020,1.249200e+18,1249200277670035457,سندھ حکومت کی جانب سے مستحقین کو ایکسپائرڈ ا...,"<a href=""https://about.twitter.com/products/tw...",True,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Sun Apr 12 08:20:11 +0000 2020,1.249251e+18,1249250950692257792,जिले के नागरिक Corona Manager- Ashoknagar एप क...,"<a href=""http://twitter.com/download/android"" ...",True,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
dfQuotedStatusPermaLink = pandas.json_normalize(dfTweets["quoted_status_permalink"])
dfQuotedStatusPermaLink = dfQuotedStatusPermaLink[dfQuotedStatusPermaLink["url"].notna()]
dfQuotedStatusPermaLink.head(5)

,url,expanded,display
2,https://t.co/0ogc9Xj7Jp,https://twitter.com/BTS_BR/status/124935288784...,twitter.com/BTS_BR/status/…
11,https://t.co/BE5WqQqVU1,https://twitter.com/ImranKhanPTI/status/124935...,twitter.com/ImranKhanPTI/s…
19,https://t.co/hK7ivE9r8y,https://twitter.com/TV9Marathi/status/12493999...,twitter.com/TV9Marathi/sta…
20,https://t.co/Gv3p7mqVXT,https://twitter.com/siasatpk/status/1249200277...,twitter.com/siasatpk/statu…
26,https://t.co/G2lGr1jtZw,https://twitter.com/projsashoknagar/status/124...,twitter.com/projsashoknaga…


### Storing IDs 
Is there a reason to retain both ID and ID str? Should we keep the numerical or string ID? For all IDs? 

From the stats, it appears that ID and ID string always match. Check a couple of different IDs from each of the objects.

In [59]:
# Let's define a function since the schema seems consistent
def fnIDMatchesIDStr(df):
    return df[pandas.to_numeric(df.id_str) == df.id]["id"].count() == len(df)

# Check Tweet IDs
print("Tweet ID and ID str match? ", fnIDMatchesIDStr(dfTweets))
print("Retweet ID and ID str match? ", fnIDMatchesIDStr(dfRetweetStatus))
print("Quoted ID and ID str match? ", fnIDMatchesIDStr(dfQuotedStatus))

Tweet ID and ID str match?  True
Retweet ID and ID str match?  True
Quoted ID and ID str match?  True


### Source Field

What is the significance of the source field? 

It seems to indicate the app client used to generate the tweet. Given the scope of the project, this field can likely be dropped.

In [69]:
with pandas.option_context('display.max_columns', None):
    print(dfTweets["source"].dropna().unique())

['<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>'
 '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>'
 '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>'
 '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>'
 '<a href="https://github.com/TwidereProject" rel="nofollow">Twidere for Android</a>'
 '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'
 '<a href="http://www.horst-karbaum.de" rel="nofollow">Buch8</a>'
 '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>'
 '<a href="http://itunes.apple.com/us/app/twitter/id409789998?mt=12" rel="nofollow">Twitter for Mac</a>'
 '<a href="http://instagram.com" rel="nofollow">Instagram</a>'
 '<a href="https://socialbee.io/" rel="nofollow">SocialBee.io v2</a>'
 '<a href="http://www.betterjobsearch.com" rel="nofollow">betterjobsearch</a>'
 '<a href="https://crowdfireapp.com" rel

### Reply to Screen Name

Does this field provide any additional information that is not otherwise available via the in reply to user? Since the author's of the tweet, i.e. user, is obviously not a complete list of users, we need the "in_reply_to_screen_name" to capture those
other's users tweets which are not in the stream.

In [108]:
dfInReplyToScreenName = dfTweets[dfTweets["in_reply_to_screen_name"].notna()]
dfInReplyToScreenName = dfInReplyToScreenName[["in_reply_to_screen_name", "in_reply_to_user_id"]]
dfInReplyToScreenName.columns = ["screen_name", "id"]
dReconcile = dfInReplyToScreenName.merge(dfUser, on="id")

print("Number of records with screen name: ", len(dfInReplyToScreenName.notna()))
print("Screen name matching with user id: ", 
      dReconcile[dReconcile["screen_name_x"] == dReconcile["screen_name_y"]]["id"].count())

Number of records with screen name:  291
Screen name matching with user id:  20


### Sensitive Tweets

What is a sensitive tweet, or tweet with sensitive link? Are these objectionable and/or offensive content or is the underlying link or web media somehow sensitive?

In [115]:
with pandas.option_context('display.max_columns', None):
    print(dfTweets[dfTweets["possibly_sensitive"] == True]["text"])

282     RT @CutestFunniest: Corona, asteroid &amp; ear...
564     Even better, you don’t waste time traveling or...
565       https://t.co/aOKtbCezjR https://t.co/QEZm6EPMPl
754     RT @comradereddy: Authentic information ...spr...
931     RT @Caryredbone: BREAKING: AOC Gets Caught LYI...
950     RT @VIVAcoid: Kabar Gembira, Semua Pasien Posi...
1019    RT @jun_makino: 火中の栗な気もしますが  https://t.co/6OmZ...
1149    RT @markyourlee: negatif corona positif sintin...
1303    conservatives are going to go out for easter t...
1360    Koko corona virus 🦠 la Nou pap bay nou pap pra...
1503    RT @kegblgnunfaedh: Efek kelamaan dirumah nega...
1748    RT @rudydagoat_: CORONA GOT NIGGAS PARANOID @ ...
Name: text, dtype: object


### Extended Entitites

Per the description in the API docs, it does not appear as if the extended entities can be used in this search application. 
So it might best to discard this field completely.

https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/extended-entities#extended-entities-object

In [128]:
dfExtendedEntities = pandas.json_normalize(dfTweets["extended_entities"])
dfExtendedEntities = dfExtendedEntities[dfExtendedEntities["media"].notna()]
dfExtendedEntities

,media
2,"[{'id': 1249403764035006465, 'id_str': '124940..."
22,"[{'id': 1249400156916781064, 'id_str': '124940..."
39,"[{'id': 1249305718592802817, 'id_str': '124930..."
63,"[{'id': 1242244315759497222, 'id_str': '124224..."
95,"[{'id': 1249404031644184577, 'id_str': '124940..."
...,...
1887,"[{'id': 1249408593297903616, 'id_str': '124940..."
1893,"[{'id': 1248980554579615745, 'id_str': '124898..."
1922,"[{'id': 1249249754757894144, 'id_str': '124924..."
1936,"[{'id': 1249408716329410560, 'id_str': '124940..."


## Part 4 - Other Objects

Beyond Tweet themselves, there are a number of other fields that bear a closer examination. These will be explored
here to determine what should be modeled in the search application and what can be safely discarded.

### Places

What are the unique place names and types? What's the difference between name and fullname? 

In [132]:
print(dfPlace["place_type"].unique())
print(dfPlace["name"].unique())
print(dfPlace["full_name"].unique())
print(dfPlace["country"].unique())

['city' 'admin' 'country']
['New Delhi' 'Punjab' 'Düsseldorf' 'Florida' 'Fairburn' 'Abuja' 'Erlen'
 'Rio de Janeiro' 'Şanlıurfa' 'Berlin' 'Hyderabad' 'Accra' 'Haflong'
 'İnegöl' 'İstanbul' 'Mendes' 'Adana' 'Hatay Merkez' 'Uttar Pradesh'
 'Porto Alegre' 'São Gonçalo' 'جمهوری اسلامی ایران' 'Yemen' 'Mumbai'
 'Bursa' 'Montana' 'Kaduna' 'Sri Lanka']
['New Delhi, India' 'Punjab, Pakistan' 'Düsseldorf, Deutschland'
 'Florida, USA' 'Fairburn, GA' 'Abuja, Nigeria' 'Erlen, Schweiz'
 'Rio de Janeiro, Brasil' 'Şanlıurfa, Türkiye' 'Berlin, Deutschland'
 'Hyderabad, India' 'Accra, Ghana' 'Haflong, India' 'İnegöl, Türkiye'
 'İstanbul, Türkiye' 'Mendes, Brasil' 'Adana, Türkiye'
 'Hatay Merkez, Hatay' 'Uttar Pradesh, India' 'Porto Alegre, Brésil'
 'São Gonçalo, Brasil' 'جمهوری اسلامی ایران' 'Yemen' 'Mumbai, India'
 'Bursa, Türkiye' 'Montana, USA' 'Kaduna, Nigeria' 'Sri Lanka']
['India' 'Pakistan' 'Deutschland' 'United States' 'Nigeria' 'Schweiz'
 'Brasil' 'Türkiye' 'Ghana' 'Turkey' 'Brésil' 'جمهوری اسل

KeyError: 'attributes'